In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

In [63]:
big_countries = ["AUS", "BRA", "CAN", "CHN", "IND", "RUS", "USA"]

yrs = range(1980, 2016+1)
yrs_s = [str(xx) for xx in yrs]

In [45]:
dr_states = pd.read_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/input_damage_distribution/natcat_drought_states_AUS-BRA-CAN-CHN-IND-RUS-USA.csv', index_col="event_id")
dr_states.head(50)

,iso3,Affected Area (Detail),affected_states,affected_states_dose
event_id,,,,
419,AUS,New South Wales,New South Wales (New South Wales),AUS.5_1
1102,AUS,"Victoria; New South Wales, Broken Hill; Queens...",Victoria (Victoria); New South Wales (New Sout...,"AUS.10_1, AUS.5_1, AUS.7_1, AUS.8_1"
1251,AUS,"South Australia; Victoria; New South Wales, Br...",South Australia (South Australia); Victoria (V...,"AUS.10_1, AUS.5_1, AUS.7_1, AUS.8_1"
5266,AUS,Queensland; South Australia,Queensland (Queensland); South Australia (Sout...,"AUS.7_1, AUS.8_1"
6406,AUS,Queensland; Western Australia; South Australia...,Queensland (Queensland); Western Australia (We...,"AUS.11_1, AUS.5_1, AUS.7_1, AUS.8_1"
8631,AUS,Queensland; New South Wales,Queensland (Queensland); New South Wales (New ...,"AUS.5_1, AUS.7_1"
9536,AUS,"New South Wales, Mo naro area; Queensland; Vic...","New South Wales (New South Wales), Mo naro are...","AUS.10_1, AUS.5_1, AUS.7_1"
9824,AUS,New South Wales,New South Wales (New South Wales),AUS.5_1
10135,AUS,"New South Wales, Shoalhaven; Northern Territor...","New South Wales (New South Wales), Shoalhaven ...","AUS.5_1, AUS.6_1, AUS.7_1"


In [21]:
dose = pd.read_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/input_damage_distribution/DoseRegions.csv')

In [58]:
tot_gdp = pd.read_csv("/home/insauer/projects/droughts/droughts/data/socio_economic/input_damage_distribution/DOSE_V2_grp_usd_2015_running_11yr_mean.csv", index_col=0)

In [22]:
natcat_damage = pd.read_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_converted/natcat_droughts_converted.csv', index_col=0)
natcat_damage

,year,begin,begin_yr,begin_mon,end,end_yr,end_mon,event,type,subtype,...,Association Type \n(Single + Main = Single-Country-Event; Country = Country data from Multi-Country-Event; Region =Multi-Country-Event; see sheet 'Read me'),Country Name,Affected Area,Affected Area (Detail),Description,Catastrophe Class,Geocoded Place,damage_constant2015USD_PPP,damage_constant2017USD_PPP,conversion_source
id,,,,,,,,,,,,,,,,,,,,,
0,1980,01/01/1980 00:00,1980,1,31/08/1982 00:00,1982,8,Drought,D,dr:Drought,...,Country,Spain,"S, C",Murcia; Andalucia; Baleares,"Lack of rain, total abstinence of rain, dry sp...",4,ESP: Murcia,4824.336569,5166.227764,GDPuc_currentUSD_to_constantXPPP
9,1980,01/01/1980 00:00,1980,1,31/01/1980 00:00,1980,1,Drought,D,dr:Drought,...,Single,Somalia,"Entire country,",n.a,"Lack of rain. Cropland damaged, livestock affe...",3,SOM:,0.069383,0.070457,GDPuc_currentUSD_to_constantXPPP
10,1980,01/01/1980 00:00,1980,1,31/12/1980 00:00,1980,12,Drought,D,dr:Drought,...,Country,Mozambique,"S, C,",Manica; Sofala; Inhambane; Gaza,"Lack of rain, dry spell, hot temperatures. Riv...",3,MOZ: Manica,193.657903,187.422649,GDPuc_constant2016MER_to_constantXPPP
11,1980,01/01/1980 00:00,1980,1,31/12/1985 00:00,1985,12,Drought,D,dr:Drought,...,Country,Gambia,"Entire country,",n.a,"Lack of rain, dry spell. Cropland damaged/dest...",3,GMB:,66.349153,66.085882,GDPuc_currentUSD_to_constantXPPP
12,1980,01/01/1980 00:00,1980,1,31/12/1982 00:00,1982,12,Drought,D,dr:Drought,...,Country,Mali,"W,","Kayes, Timbuktu","Lack of rain, dry spell, hot temperatures. Cro...",3,MLI: Kayes,125.630406,132.248623,GDPuc_currentUSD_to_constantXPPP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29778,2016,01/07/2016 00:00,2016,7,17/08/2016 00:00,2016,8,Drought,D,dr:Drought,...,Single,France,"W, C, S,","Eure-et-Loir, Indre, Loire-Atlantique, Vendée,...","Dry conditions, lack of rain. Water restrictio...",0,"FRA: Loiré, Loiré",0.111133,0.117923,GDPuc_currentUSD_to_constantXPPP
29936,2016,01/08/2016 00:00,2016,8,05/09/2016 00:00,2016,9,Drought,D,dr:Drought,...,Single,China,"N,",Gansu,"Lack of rain. Crops (esp. corn, potato) destro...",3,CHN: Gansu Sheng,761.754029,744.912423,GDPuc_currentUSD_to_constantXPPP
30097,2016,01/09/2016 00:00,2016,9,24/09/2016 00:00,2016,9,Drought,D,dr:Drought,...,Single,China,"C,",Hubei,"Lack of rain. Shortage of drinking water. >1,2...",2,CHN: Hubei,101.567204,99.321656,GDPuc_currentUSD_to_constantXPPP


In [23]:
natcat_damage['damage_constant2015USD_PPP'] = natcat_damage['damage_constant2015USD_PPP'] * 1e6

In [25]:
natcat_new

,event_id,yrmon_begin,yrmon_end,subtype,damage_constant2015USD_PPP,iso3,state,affected,affected_area,affected_states,total_gdp,total_gdp_weight,total_gdp_damage


In [60]:

natcat_new = pd.DataFrame(columns=[
        "event_id", "yrmon_begin", "yrmon_end", "subtype", "damage_constant2015USD_PPP", "iso3", "state", "affected",
        "affected_area", "affected_states", 'total_gdp', 'total_gdp_weight', 'total_gdp_damage'])
for ev in natcat_damage.index:
    print(ev)
    yr_begin, mon_begin, yr_end, mon_end = natcat_damage.loc[ev, ["begin_yr", "begin_mon", "end_yr", "end_mon"]]
    yrmon_ev = [f"{yr_begin}{str(mon_begin).zfill(2)}", f"{yr_end}{str(mon_end).zfill(2)}"]
    data_ev = pd.DataFrame(index=[0], columns=natcat_new.columns)
        # do not distribute the damages, just take the national damage vs. the national indicator.
    iso3 = natcat_damage.loc[ev, "iso3"]    
    data_ev.loc[:, "event_id"] = ev
    data_ev.loc[:, ["yrmon_begin", "yrmon_end"]] = yrmon_ev
    data_ev.loc[:, ["subtype", "damage_constant2015USD_PPP", "iso3"]] = natcat_damage.loc[ev, ["subtype", "damage_constant2015USD_PPP", "iso3"]].values
    data_ev.loc[:, "state"] = iso3
    data_ev.loc[:, "affected"] = True
    data_ev.loc[:, "affected_area"] = "national"
    data_ev.loc[:, "affected_states"] = "national"
    data_ev.loc[:, "total_gdp_weight"] = 1
    data_ev.loc[:, "total_gdp_damage"] = natcat_damage.loc[ev, "damage_constant2015USD_PPP"]
    natcat_new = pd.concat([natcat_new, data_ev], ignore_index=True)
    if natcat_damage.loc[ev, "iso3"] in big_countries:
        states_aff = dr_states.loc[ev, "affected_states_dose"].split(", ") # affected states, dose codes.
        codes_iso = dose.GID_1[dose.GID_0 == iso3].tolist()
        states_naff = [xx for xx in codes_iso if xx not in states_aff]
        data_ev = pd.DataFrame(index=codes_iso, columns=natcat_new.columns)
        data_ev.loc[:, "state"] = codes_iso
        data_ev.loc[:, "event_id"] = ev
        data_ev.loc[:, ["yrmon_begin", "yrmon_end"]] = yrmon_ev
        data_ev.loc[:, ["subtype", "damage_constant2015USD_PPP", "iso3"]] = natcat_damage.loc[ev, ["subtype", "damage_constant2015USD_PPP", "iso3"]].values
        data_ev.loc[:, "affected_area"] = "states"
        data_ev.loc[:, "affected_states"] = ", ".join(states_aff)
        data_ev.loc[states_aff, "affected"] = True
        data_ev.loc[states_naff, "affected"] = False
        
        ev_yrs = [str(xx) for xx in range(yr_begin, yr_end+1)]
        # there might be some missing data -> this state is set to weight = 0.
        gdp_states = tot_gdp.loc[:, ev_yrs].mean(axis=1).reindex(index=states_aff + states_naff)
        data_ev.loc[:, 'tot_gdp'] = gdp_states
        weights = gdp_states.loc[states_aff].div(gdp_states.loc[states_aff].sum())
        # set weights 0 if they are NaN.
        weights[weights.isnull()] = 0
        # if there is only one weight -> it is 1.
        if len(weights) == 1:
            weights[:] = 1 # is not calculated as 1 if it is NaN.
        
        data_ev.loc[states_aff, "tot_gdp_weight"] = weights.values
        data_ev.loc[:, "tot_gdp_damage"] = data_ev.loc[:, "tot_gdp_weight"].multiply(natcat_damage.loc[ev, "damage_constant2015USD_PPP"])
        
        if weights.sum() == 0:
            print("\nfor", iso3, "tot_gdp", "and event", ev, "the weights are all 0, even though a state was affected")
            print("  len(weights) =", len(weights))
            print("  affected states:", states_aff)
            print(data_ev.loc[states_aff, ["state", "tot_gdp", "tot_gdp_weight", "tot_gdp_damage"]])
        natcat_new = pd.concat([natcat_new, data_ev], ignore_index=True)

0
9
10
11
12
13
14
15
16
81
158
159
160
161
162
163
166
168
329
360
418
419
420
421
422
455
456
457
581
617
665
702
751
824
825
826
889
928
931
982
1019
1020
1024
1062
1102
1103
1145
1175
1176
1251
1255
1256
1257
1258
1303
1304
1305
1306
1309
1310
1378
1380
1382
1383
1419
1421
1422
1423
1452
1453
1454
1488
1620
1621
1648
1680
1681
1711
1716
1717
1718
1719
1752
1860
1923
1977
2042
2055
2057
2238
2572
2604
2608
2609
2648
2681
2879
2895
2896


/tmp/ipykernel_4451/3812056559.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  natcat_new = pd.concat([natcat_new, data_ev], ignore_index=True)
/tmp/ipykernel_4451/3812056559.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  natcat_new = pd.concat([natcat_new, data_ev], ignore_index=True)
/tmp/ipykernel_4451/3812056559.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA 

2897
2898
2899
2900
2901
2902
2984
3023
3025
3053
3093
3094
3150
3151
3278
3316
3423
3451
3452
3453
3454
3457
3458
3459
3460
3461
3462
3555
3557
3558
3594
3601
3622
3632
3674
3786
3787
3855
3858
3914
3915
4010
4011
4047
4059
4061
4096
4141
4175
4211
4316
4317
4389
4539
4540
4567
4604
4614
4615
4616
4667
4808
4809
4855
4919
4983
4984
5040
5137
5189
5221
5225
5226
5265
5266
5267
5268
5270
5309
5334
5335
5336
5340
5341
5371
5372
5463
5513
5518
5601
5689
5777
5778
5826
5835
5841
5923
5971
6005
6007
6052
6053
6054
6123
6190
6191
6258
6351
6406
6446
6453
6568
6618
6619
6620
6673
6749
6751
6752
6753
6754
6755
6756
6757
6758
6759
6824
6917
6989
7100
7210
7216
7352
7401
7451
7452
7453
7458
7459
7461
7462
7493
7515
7518
7522
7523
7524
7525
7526
7527
7733
7734
7902
7903
7935
7940
7941
8075
8079
8125
8126
8129
8172
8175
8176
8308
8311
8462
8463
8516
8574
8621
8631
8696
8697
8698
8749
8752
8838
8841
8957
9065
9066
9253
9254
9314
9375
9376
9418
9420
9472
9473
9474
9535
9536
9539
9540
9541
9543
9544


In [98]:
natcat_new.to_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_distributed/natcat_converted_distributed.csv')

In [61]:
natcat_new.loc[natcat_new['event_id']==419]

,event_id,yrmon_begin,yrmon_end,subtype,damage_constant2015USD_PPP,iso3,state,affected,affected_area,affected_states,total_gdp,total_gdp_weight,total_gdp_damage,tot_gdp,tot_gdp_weight,tot_gdp_damage
99,419,198101,198112,dr:Drought,193527684.476782,AUS,AUS,True,national,national,NaN,1,193527684.476782,NaN,NaN,NaN
100,419,198101,198112,dr:Drought,193527684.476782,AUS,AUS.10_1,False,states,AUS.5_1,NaN,NaN,NaN,77423264087.079529,NaN,NaN
101,419,198101,198112,dr:Drought,193527684.476782,AUS,AUS.11_1,False,states,AUS.5_1,NaN,NaN,NaN,25345671295.456329,NaN,NaN
102,419,198101,198112,dr:Drought,193527684.476782,AUS,AUS.1_1,False,states,AUS.5_1,NaN,NaN,NaN,NaN,NaN,NaN
103,419,198101,198112,dr:Drought,193527684.476782,AUS,AUS.2_1,False,states,AUS.5_1,NaN,NaN,NaN,5478141329.744714,NaN,NaN
104,419,198101,198112,dr:Drought,193527684.476782,AUS,AUS.3_1,False,states,AUS.5_1,NaN,NaN,NaN,NaN,NaN,NaN
105,419,198101,198112,dr:Drought,193527684.476782,AUS,AUS.4_1,False,states,AUS.5_1,NaN,NaN,NaN,NaN,NaN,NaN
106,419,198101,198112,dr:Drought,193527684.476782,AUS,AUS.5_1,True,states,AUS.5_1,NaN,NaN,NaN,98179229904.220474,1.0,193527684.476782
107,419,198101,198112,dr:Drought,193527684.476782,AUS,AUS.6_1,False,states,AUS.5_1,NaN,NaN,NaN,2857512822.734621,NaN,NaN
108,419,198101,198112,dr:Drought,193527684.476782,AUS,AUS.7_1,False,states,AUS.5_1,NaN,NaN,NaN,40319209174.049309,NaN,NaN


In [97]:
yrmons = pd.Series([int(f"{yr}{str(mon).zfill(2)}") for yr in yrs for mon in range(1, 12+1)], dtype=int)
regions = sorted(natcat_new.state.unique())
dams_ts = pd.DataFrame(0.0, index=regions, columns=yrs)
for row in natcat_new.index:
        
    data_act = natcat_new.loc[row, :]
    # damages distributed to years by equally weighting with event months.
    # put the damage div. number of event months into a pd.Series with the event yrmons as index.
    ev_dams = pd.Series(index=yrmons[(yrmons>=int(data_act.yrmon_begin)) & (yrmons<=int(data_act.yrmon_end))], dtype=float)
    ev_dams[:] = data_act["total_gdp_damage"]/len(ev_dams)
    for yr in sorted(set([str(xx)[:4] for xx in ev_dams.index])): # each event year, as string.
        print(yr)
        dams_ts.loc[data_act.state, int(yr)] = dams_ts.loc[data_act.state, int(yr)] + \
            ev_dams.loc[[xx for xx in ev_dams.index if str(xx)[:4] == yr]].sum()

1980
1981
1982
1980
1980
1980
1981
1982
1983
1984
1985
1980
1981
1982
1980
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1980
1981
1982
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1981
1982
1980
1981
1982
1983
1984
1985
1980
1981
1982
1983
1980
1981


In [96]:
dams_ts

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
AFG,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.501701e+08,3.751418e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
AGO,0.000000e+00,1.857970e+08,0.000000e+00,0.000000e+00,0.000000e+00,4.768621e+08,0.000000e+00,0.0,0.000000e+00,1.204557e+08,...,0.000000e+00,8.797946e+06,0.000000e+00,0.000000e+00,0.000000e+00,8.632569e+06,5.844557e+07,1.461139e+07,5.426166e+08,8.500125e+07
ALB,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,2.522113e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
ARG,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,6.069764e+07,0.000000e+00,...,4.740772e+08,4.740772e+08,4.740772e+08,1.185193e+08,2.578278e+09,1.289139e+09,4.705648e+08,0.000000e+00,0.000000e+00,0.000000e+00
ARM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VUT,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.050820e+04,0.000000e+00
WSM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.394648e+04,0.000000e+00
ZAF,2.146065e+07,0.000000e+00,3.720068e+09,3.720068e+08,0.000000e+00,0.000000e+00,1.007422e+08,0.0,2.100379e+07,0.000000e+00,...,5.569649e+07,3.037990e+07,0.000000e+00,8.172339e+06,0.000000e+00,0.000000e+00,7.369260e+06,1.842315e+06,3.064411e+09,1.180437e+08
ZMB,0.000000e+00,0.000000e+00,1.333077e+08,1.333077e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.563269e+07,2.697239e+08


In [ ]:
dams_ts.to_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_distributed/natcat_converted_distributed.csv')